In [2]:
!pip install beautifulsoup4
!pip install jq
!pip install -U transformers huggingface_hub
!pip install -q bitsandbytes torch accelerate
!pip install langchain faiss-cpu sentence-transformers langchainhub
!pip install fastapi nest-asyncio pyngrok uvicorn

In [2]:
import requests
import warnings
warnings.simplefilter('ignore')
import re
from datetime import date
from langchain import hub
import multiprocessing
from bs4 import BeautifulSoup
from langchain.document_transformers import EmbeddingsRedundantFilter
from tqdm import tqdm
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings , HuggingFaceBgeEmbeddings
from transformers import AutoModelForCausalLM , AutoTokenizer , BitsAndBytesConfig , pipeline ,VitsModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser , JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_community.llms import HuggingFaceEndpoint
from huggingface_hub import login
from langchain_community.document_loaders import JSONLoader
from langchain.document_transformers import LongContextReorder
from langchain.chains import RetrievalQA
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import torch
from getpass import getpass
import os
import json

2024-03-27 15:24:36.518771: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 15:24:36.518832: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 15:24:36.520216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
login()

In [6]:
home_page_url = "https://ticka.world/shop/"
sub_pages_url="https://ticka.world/shop/page/"
home_page = requests.get(home_page_url)
sub_page = requests.get(home_page_url)
soup = BeautifulSoup(home_page.content, "html.parser")

In [7]:
results = soup.find("div",class_="products layout-grid grid-columns-4 grid-columns--tablet3 grid-columns--mobile2")

In [4]:
def preprocessProductName(productName):
    productName= productName.lower().replace('–','').replace('  ',' ').split(' ')
    sep='-'
    res=sep.join(productName)
    return res

In [9]:
def removeWhiteSpaces(ch):
    return " ".join(ch.split())
def getProductsInfo(productName):
     productUrl=f'https://ticka.world/product/{productName}'
    page=requests.get(productUrl)
    if page.status_code !=200 :
        return "",""
    scoup=BeautifulSoup(page.content,"html.parser")
    infoDiv=scoup.find("ul",class_="list-unstyled")
    details=infoDiv.find_all("span",class_="details")
    rate=infoDiv.find("li",class_="clearfix")
    prefix=["Store Name : ","vendor Name : ","Adresse : "]
    store_Name=removeWhiteSpaces(details[0].text)
    rate=removeWhiteSpaces(rate.text)
    return store_Name,rate

In [10]:
def check_stroe_if_exist(data,store_Name):
    store_index=[index for index , name in enumerate(data) if name["Store Name : "]==store_Name ]
    return store_index

In [11]:
def fetchItems(results,data):
    job_elements = results.find_all("div",class_="product-info")
    for i  in tqdm(range(len(job_elements))) :
        title_element = job_elements[i].find("h2", class_="woocommerce-loop-product__title")
        price=job_elements[i].find("span",class_="woocommerce-Price-amount amount")
        price=re.sub('[^0-9.]','',str(price))+" TND"
        title_element=preprocessProductName(title_element.text)
        store_name,rate=getProductsInfo(title_element)
        check_store=check_stroe_if_exist(data,store_name)
        if(len(check_store)==0):
            data.append({"Store Name : ":store_name,"rate : ":rate,"Products : ":[]})
            check_store=[len(data)-1]
        data[check_store[0]]["Products : "].append( {"product":title_element,"price ":price})
    return data

In [12]:
def getItems(data):
    for num_page in range(1,20):
        sub_page = requests.get(sub_pages_url+str(num_page))
        scoup = BeautifulSoup(sub_page.content, "html.parser")
        results = scoup.find("div",class_="products layout-grid grid-columns-4 grid-columns--tablet3 grid-columns--mobile2")
        data=fetchItems(results,data)
    return data

In [13]:
data=getItems([])

100%|██████████| 20/20 [01:14<00:00,  3.70s/it]


In [14]:
def getTickaInformations(tickaInformation):
    landingPageUrl="https://ticka.world/"
    page=requests.get(landingPageUrl)
    scoup = BeautifulSoup(page.content,"html.parser")
    questions=scoup.find_all("a",class_="haru-accordion-title")
    result=scoup.find_all("div",class_="haru-tab-content haru-clearfix")
    activeItem=scoup.find("div",class_="haru-tab-content haru-clearfix active")
    full_text="Question : "+questions[0].text+"\nReponse : "+activeItem.find("span").text
    tickaInformation.append({"content":full_text})
    for i in tqdm(range(len(result))):
        full_text="Question : "+questions[i+1].text+"\nReponse : "+result[i].find("span").text
        tickaInformation.append({"content":full_text})
    return tickaInformation

In [15]:
tickaInformation=getTickaInformations([])

100%|██████████| 11/11 [00:00<00:00, 18008.33it/s]


In [17]:
data.append({"Informations":tickaInformation})
data={"StoreInformations":data}

In [18]:
data=json.dumps(data)

In [19]:
with open("data.json","w") as file :
    file.write(data)

In [24]:
def getTokenizer(model_repo):
    tokenizer=AutoTokenizer.from_pretrained(model_repo)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    return tokenizer

def uploadModel(model_repo):
    tokenizer = getTokenizer(model_repo)
    bnb_config = BitsAndBytesConfig(load_in_4bit=True)
    model=AutoModelForCausalLM.from_pretrained(model_repo,quantization_config=bnb_config)
    llm=pipeline(
        model=model,
        task="text-generation",
        tokenizer=tokenizer,
        temperature=0.6,
        return_full_text=False,
        do_sample=True,
        max_new_tokens=512,
    )
    mistral = HuggingFacePipeline(pipeline=llm)
    return mistral

In [4]:
def loadJsonData():
    loader = JSONLoader(
        file_path='/kaggle/input/datajsonnew/data.json',
        jq_schema='.StoreInformations[]',
        text_content=False
    )
    return loader.load()

In [5]:
def split_text():
    json_data = loadJsonData()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200
    )
    doc=text_splitter.split_documents(json_data)
    return doc

In [6]:
def getRetiver(embed):
    doc=split_text()
    db=FAISS.from_documents(doc,embed)
    retiver=db.as_retriever(
        search_type="similarity", search_kwargs={"k": 5, "include_metadata": True}
    )
    return retiver

In [7]:
def load_Embeddings():
    embed1=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
    embed2=HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-dot-v1")
    filter_embed= HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-en",model_kwargs={"device": "cpu"},encode_kwargs={"normalize_embeddings": False},)
    return embed1 ,embed2,filter_embed

In [8]:
def createRetriever():
    embed1 ,embed2,filter_embed=load_Embeddings()
    retriever1=getRetiver(embed1)
    retriever2=getRetiver(embed2)
    filter = EmbeddingsRedundantFilter(embeddings=filter_embed) #Delete Redundant docs
    return retriever1,retriever2,filter
        

In [9]:
def merge_retrievers():
    retriever1,retriever2,filter = createRetriever()
    reordering = LongContextReorder() #tackle long context llms problems
    lotr=MergerRetriever(retrievers=[retriever1,retriever2])
    pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])
    retriever = ContextualCompressionRetriever(
        base_compressor=pipeline,
        base_retriever=lotr,
        search_kwargs={"k": 5, "include_metadata": True},
    )
    return retriever

In [10]:
def getCompletion(model,question):
    retiver= merge_retrievers()
    prompt = hub.pull("rlm/rag-prompt-mistral")
    qa_chain = RetrievalQA.from_chain_type(
        model, retriever=retiver, chain_type_kwargs={"prompt": prompt}
    )
    result=qa_chain({"query":question})
    return result["result"]

# Mistral

In [19]:
def load_Model():
    model_repo ="mistralai/Mistral-7B-Instruct-v0.2"
    model=uploadModel(model_repo)
    return model

In [20]:
 model =load_Model()

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# FastAPI

In [21]:
app = FastAPI()

In [22]:
@app.get('/')
async def root():
    return {'hello': 'To ask question go to /ask/{here pass your question}'}
@app.get('/ask/{question}')
async def ask_question(question : str):
    global model
    result = getCompletion(model,question)
    return {"Answer ":result}

In [23]:
port = 8000
ngrok_tunnel = ngrok.connect(port)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=port)

Public URL: https://b72f-34-80-79-130.ngrok-free.app


INFO:     Started server process [349]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     102.159.218.228:0 - "GET / HTTP/1.1" 200 OK
INFO:     102.159.218.228:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:     102.159.218.228:0 - "GET /ask/give%20me%202%20hoodies%20product%20 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [349]
